<a href="https://colab.research.google.com/github/PremGorecki/MachineLearning/blob/main/07_lasy_losowe_zadanie8_i_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" 
8 Wczytaj zbiór danych MNIST (zob. rozdział 3.) i podziel go na podzbiory uczący, walidacyjny
i testowy (w stosunku 50 000 : 10 000 : 10 000). Następnie wytrenuj różne klasyfikatory, np. losowego
lasu, Extra-Trees czy SVM. Teraz spróbuj połączyć je w jeden zespół (korzystając z klasyfikatora
głosowania większościowego lub miękkiego), który będzie osiągał lepsze wyniki wobec
zestawu walidacyjnego w stosunku do pojedynczych klasyfikatorów. Po uzyskaniu takiego
modelu wypróbuj go na zbiorze testowym. O ile lepiej sprawuje się ten zespół w porównaniu do
poszczególnych klasyfikatorów?

Rozwiązania tych ćwiczeń znajdziesz w dodatku A.
"""

In [ ]:
"""
9. Uruchom poszczególne klasyfikatory utworzone w poprzednim ćwiczeniu w celu wyliczenia
prognoz dla zbioru walidacyjnego, a następnie utwórz nowy zbiór danych zawierający te predykcje:
każdy przykład uczący ma stanowić wektor przechowujący zbiór prognoz pochodzących z pojedynczych
klasyfikatorów dla obrazu, natomiast wartością docelową niech będzie klasa obrazu

Wytrenuj klasyfikator na tym nowym zestawie danych uczących. Gratulacje! Właśnie udało
Ci się wyuczyć mikser, który wraz z klasyfikatorami tworzy zespół kontaminacji. Sprawdź teraz
wydajność tego zespołu wobec danych testowych. Dla każdego obrazu w zbiorze testowym wylicz
prognozy za pomocą wszystkich klasyfikatorów, a następnie prześlij wyniki do miksera, aby uzyskać
przewidywania zespołu. Porównaj wyniki z utworzonym wcześniej klasyfikatorem głosującym

Rozwiązania tych ćwiczeń znajdziesz w dodatku A.

"""

In [11]:
# Zadanie 8

import pandas as pd
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [ ]:
X = mnist.data
y = mnist.target

In [ ]:
# Nie trzeba ich mieszać bo są pomieszane


In [ ]:
#Podział na zbiory uczący, validacyjny, testowy
X_train, X_valid, X_test = X[:50000], X[50000:60000], X[60000:]
y_train, y_valid, y_test = y[:50000], y[50000:60000], y[60000:]

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)


(50000, 784) (10000, 784) (10000, 784)
(50000,) (10000,) (10000,)


In [13]:
# Inny podział
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=69)

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(50000, 784) (10000, 784) (10000, 784)
(50000,) (10000,) (10000,)


In [14]:
#Trenujemy różne klasyfikatory
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.ensemble import ExtraTreesClassifier   #Extra Trees
from sklearn.svm import LinearSVC                   #SVC
# spróbować na tym from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier    #MLP

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=69)
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=69)
svc_clf = LinearSVC(random_state=69)
mlp_clf = MLPClassifier(random_state=69)


In [15]:
#Trenujemy

estimators = [rnd_clf, ext_clf, svc_clf, mlp_clf]

for estimator in estimators:
  print("Trenowany estymator: ", estimator)
  estimator.fit(X_train, y_train)

Trenowany estymator:  RandomForestClassifier(random_state=69)
Trenowany estymator:  ExtraTreesClassifier(random_state=69)
Trenowany estymator:  LinearSVC(random_state=69)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Trenowany estymator:  MLPClassifier(random_state=69)


In [16]:
#Jaki wyniki osiągnęły

for estimator in estimators:
  print(estimator, ": ", estimator.score(X_val, y_val))

RandomForestClassifier(random_state=69) :  0.969
ExtraTreesClassifier(random_state=69) :  0.9708
LinearSVC(random_state=69) :  0.8723
MLPClassifier(random_state=69) :  0.9646


In [19]:
#Większościowy twardy
from sklearn.ensemble import VotingClassifier

hard_voting_clf = VotingClassifier(
        estimators=[('rd',  rnd_clf), 
                    ('et',  ext_clf), 
                    ('svc', svc_clf),
                    ('mlp', mlp_clf)],
        voting='hard')
hard_voting_clf.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rd', RandomForestClassifier(random_state=69)),
                             ('et', ExtraTreesClassifier(random_state=69)),
                             ('svc', LinearSVC(random_state=69)),
                             ('mlp', MLPClassifier(random_state=69))])

In [20]:
print(hard_voting_clf, ": ", hard_voting_clf.score(X_val, y_val))

VotingClassifier(estimators=[('rd', RandomForestClassifier(random_state=69)),
                             ('et', ExtraTreesClassifier(random_state=69)),
                             ('svc', LinearSVC(random_state=69)),
                             ('mlp', MLPClassifier(random_state=69))]) :  0.9714


In [27]:
#Większościowy miękki
soft_voting_clf = VotingClassifier(
estimators=[('rd',  rnd_clf), 
            ('et',  ext_clf), 
          #  ('svc', svc_clf,),
            ('mlp', mlp_clf)],
voting='soft')
soft_voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('rd', RandomForestClassifier(random_state=69)),
                             ('et', ExtraTreesClassifier(random_state=69)),
                             ('mlp', MLPClassifier(random_state=69))],
                 voting='soft')

In [28]:
print(soft_voting_clf, ": ", soft_voting_clf.score(X_val, y_val))

VotingClassifier(estimators=[('rd', RandomForestClassifier(random_state=69)),
                             ('et', ExtraTreesClassifier(random_state=69)),
                             ('mlp', MLPClassifier(random_state=69))],
                 voting='soft') :  0.9701


In [29]:
# Najlepszy jest VotingClassifier z HARD głosowanie to sprawdźmy go na danych testowych

print(hard_voting_clf, ": ", hard_voting_clf.score(X_test, y_test))



VotingClassifier(estimators=[('rd', RandomForestClassifier(random_state=69)),
                             ('et', ExtraTreesClassifier(random_state=69)),
                             ('svc', LinearSVC(random_state=69)),
                             ('mlp', MLPClassifier(random_state=69))]) :  0.968


In [30]:
# Pozostałe na danych testowych
for estimator in estimators:
  print(estimator, ": ", estimator.score(X_test, y_test))

RandomForestClassifier(random_state=69) :  0.9656
ExtraTreesClassifier(random_state=69) :  0.9691
LinearSVC(random_state=69) :  0.8737
MLPClassifier(random_state=69) :  0.9598
